In [120]:
from lxml import etree
import pandas as pd
from typing import Dict, List, Optional, Any
from itertools import product
import re
import xml.etree.ElementTree as ET
from collections import defaultdict
from rapidfuzz.fuzz import ratio

In [121]:
file_path = 'data/KC_Thesaurus_all_2025-03-21.xml'

# Structuur xmls

In [122]:
# KC_Thesaurus_all_2025-03-21.
kc_all = 'data/KC_Thesaurus_all_2025-03-21.xml'
kc_collect = 'data/KC_Thesaurus_collect_2025-03-21.xml'

def collect_structure(elem, path="", structure=None):
    if structure is None:
        structure = defaultdict(set)
    current_path = f"{path}/{elem.tag}"
    structure[path].add(elem.tag)
    for child in elem:
        collect_structure(child, current_path, structure)
    return structure

tree = ET.parse(kc_all)
root = tree.getroot()

structure = collect_structure(root)
for parent, children in structure.items():
    print(f"{parent} -> {children}")


 -> {'adlibXML'}
/adlibXML -> {'diagnostic', 'recordList'}
/adlibXML/recordList -> {'record'}
/adlibXML/recordList/record -> {'narrower_term', 'text_for_OPAC', 'notes', 'input.name', 'use.lref', 'related_term', 'use', 'input.time', 'term.type', 'source', 'input.source', 'narrower_term.lref', 'Edit', 'input.date', 'broader_term', 'equivalent_term.lref', 'scope_note', 'used_for', 'term.status', 'priref', 'related_term.lref', 'history_note', 'equivalent_term', 'use_count', 'used_for.lref', 'source_link', 'term', 'broader_term.lref', 'term.number'}
/adlibXML/recordList/record/Edit -> {'edit.notes', 'edit.date', 'edit.time', 'edit.name', 'edit.source'}
/adlibXML/recordList/record/term.type -> {'text'}
/adlibXML/recordList/record/narrower_term -> {'notes', 'input.name', 'related_term', 'input.time', 'term.type', 'source', 'input.source', 'narrower_term.lref', 'Edit', 'input.date', 'broader_term', 'equivalent_term.lref', 'scope_note', 'used_for', 'term.status', 'related_term.lref', 'history_n

In [123]:
def collect_structure(elem, path="", structure=None):
    if structure is None:
        structure = defaultdict(set)
    current_path = f"{path}/{elem.tag}"
    structure[path].add(elem.tag)
    for child in elem:
        collect_structure(child, current_path, structure)
    return structure

def structures_equal(file1, file2):
    def get_structure_dict(file_path):
        tree = ET.parse(file_path)
        root = tree.getroot()
        return dict(collect_structure(root))

    return get_structure_dict(file1) == get_structure_dict(file2)


In [124]:
print(structures_equal(kc_all, kc_collect))  # True | False voor gelijke structuren


True


In [125]:
tree = ET.parse(kc_collect)
root = tree.getroot()

structure = collect_structure(root)
for parent, children in structure.items():
    print(f"{parent} -> {children}")

 -> {'adlibXML'}
/adlibXML -> {'diagnostic', 'recordList'}
/adlibXML/recordList -> {'record'}
/adlibXML/recordList/record -> {'narrower_term', 'text_for_OPAC', 'notes', 'input.name', 'use.lref', 'related_term', 'use', 'input.time', 'term.type', 'source', 'input.source', 'narrower_term.lref', 'Edit', 'input.date', 'broader_term', 'equivalent_term.lref', 'scope_note', 'used_for', 'term.status', 'priref', 'related_term.lref', 'history_note', 'equivalent_term', 'use_count', 'used_for.lref', 'source_link', 'term', 'broader_term.lref', 'term.number'}
/adlibXML/recordList/record/Edit -> {'edit.notes', 'edit.date', 'edit.time', 'edit.name', 'edit.source'}
/adlibXML/recordList/record/term.type -> {'text'}
/adlibXML/recordList/record/narrower_term -> {'notes', 'input.name', 'related_term', 'input.time', 'term.type', 'source', 'input.source', 'narrower_term.lref', 'Edit', 'input.date', 'broader_term', 'equivalent_term.lref', 'scope_note', 'used_for', 'term.status', 'related_term.lref', 'history_n

# Analyse KC

In [126]:
def ensure_list(x: Any) -> List:
    """Ensure that x is returned as a list."""
    if x is None:
        return []
    return x if isinstance(x, list) else [x]

def get_text(element: etree._Element, path: str, default: Optional[str] = None) -> Optional[str]:
   # XPath '/text()' to extract the text node.
    found = element.xpath(path)
    return found[0] if found else default

def get_attr(element: etree._Element, attr: str, default: Optional[str] = None) -> Optional[str]:
    # Krijg attribuut per element
    return element.get(attr, default)

def parse_elements(element: etree._Element, path: str, processor) -> List[Dict]:
    #generieke parser
    elements = ensure_list(element.xpath(path))
    return [processor(el) for el in elements]

def process_term(element: etree._Element) -> Dict:
    """
    Process a term.type or term.status element.
    Returns a dict with the option, value, and a list of texts (each with text and language).
    """
    texts = parse_elements(element, 'text', lambda t: {
        'text': t.text,
        'language': get_attr(t, 'language')
    })
    return {
        'type': get_attr(element, 'option'),
        'value': get_attr(element, 'value'),
        'texts': texts
    }

def process_linked_term(element: etree._Element) -> Dict:
    """Process a linked term (e.g. broader_term or used_for)."""
    return {
        'linkref': get_attr(element, 'linkref'),
        'linkfield': get_attr(element, 'linkfield'),
        'linkreffield': get_attr(element, 'linkreffield'),
        'linkdb': get_attr(element, 'linkdb'),
        'term': get_text(element, 'term/text()'),
        'term_number': get_attr(element, 'priref') # % nieuw
    }

def expand_term_data(terms: List[Dict], prefix: str) -> List[Dict]:
    """
    Expand term data so that for each term element we create a separate row per text.
    The resulting dict uses keys with the given prefix.
    """
    expanded = []
    for term in terms:
        texts = term.get('texts') or []
        if not texts:
            expanded.append({
                f'{prefix}option': term.get('type'),
                f'{prefix}value': term.get('value'),
                f'{prefix}text': None,
                f'{prefix}language': None
            })
        else:
            for text in texts:
                expanded.append({
                    f'{prefix}option': term.get('type'),
                    f'{prefix}value': term.get('value'),
                    f'{prefix}text': text.get('text'),
                    f'{prefix}language': text.get('language')
                })
    return expanded if expanded else [{}]

def expand_linked_terms(terms: List[Dict], prefix: str) -> List[Dict]:
    """Expand linked term data to a list of dicts with keys prefixed."""
    if not terms:
        return [{}]
    return [{
        f'{prefix}linkref': term.get('linkref'),
        f'{prefix}linkfield': term.get('linkfield'),
        f'{prefix}linkreffield': term.get('linkreffield'),
        f'{prefix}linkdb': term.get('linkdb'),
        f'{prefix}term': term.get('term'),
        f'{prefix}term_number': term.get('term_number') # % nieuw
    } for term in terms]

def process_record(record: etree._Element) -> List[Dict]:
    """
    Process a single record element into a list of flattened rows.
    This function collects base fields, edit data, term type/status data, and linked term data,
    then generates one row per combination.
    """
    base = {
        'priref': get_text(record, 'priref/text()'),
        'use_count': get_text(record, 'use_count/text()'),
        'main_term': get_text(record, 'term/text()'),
        'term_number': get_text(record, 'term.number/text()'),
        'source': get_text(record, 'source/text()'),
        'source_link': get_text(record, 'source_link/text()'),
        'scope_note': get_text(record, 'scope_note/text()'),
        'input_date': get_text(record, 'input.date/text()'),
        'input_name': get_text(record, 'input.name/text()'),
        'input_time': get_text(record, 'input.time/text()'),
        'input_source': get_text(record, 'input.source/text()'),
        'narrower_term_lrefs': '|'.join(record.xpath('narrower_term.lref/text()')) or None,
        'broader_term_lrefs': '|'.join(record.xpath('broader_term.lref/text()')) or None,
        'used_for_lrefs': '|'.join(record.xpath('used_for.lref/text()')) or None,
    }
    
    edits = parse_elements(record, 'Edit', lambda e: {
        'edit_date': get_text(e, 'edit.date/text()'),
        'edit_name': get_text(e, 'edit.name/text()'),
        'edit_time': get_text(e, 'edit.time/text()'),
        'edit_source': get_text(e, 'edit.source/text()'),
        'edit_notes': get_text(e, 'edit.notes/text()')
    })
    term_types = parse_elements(record, 'term.type', process_term)
    term_statuses = parse_elements(record, 'term.status', process_term)
    broader_terms = parse_elements(record, 'broader_term', process_linked_term)
    narrower_terms = parse_elements(record, 'narrower_term', process_linked_term)  # % nieuw
    used_for_terms = parse_elements(record, 'used_for', process_linked_term)
    
    # Use itertools.product to generate all combinations.
    combinations = product(
        edits or [{}],
        expand_term_data(term_types, prefix='term_type_'),
        expand_term_data(term_statuses, prefix='status_'),
        expand_linked_terms(broader_terms, prefix='broader_'),
        expand_linked_terms(narrower_terms, prefix='narrower_'),  # % nieuw
        expand_linked_terms(used_for_terms, prefix='used_for_')
    )
    
    rows = []
    for combo in combinations:
        edit_data, type_data, status_data, broader_data, narrower_data, used_for_data = combo  # % aangepast
        row = {**base, **edit_data, **type_data, **status_data, **broader_data, **narrower_data, **used_for_data} # % aangepast
        rows.append(row)
    return rows

def xml_to_dataframe(file_path: str) -> pd.DataFrame:
    """
    Parse the XML file and convert it to a pandas DataFrame with all attributes.
    """
    tree = etree.parse(file_path)
    records = tree.xpath('//record')
    data = []
    for record in records:
        data.extend(process_record(record))
    return pd.DataFrame(data)

# Usage
df = xml_to_dataframe(file_path)

# Define the preferred final column ordering
final_columns = [
    'priref', 'use_count', 'main_term', 'term_number',
    'source', 'source_link', 'scope_note',
    'input_date', 'input_name', 'input_time', 'input_source',
    'narrower_term_lrefs', 'broader_term_lrefs', 'used_for_lrefs',
    'edit_date', 'edit_name', 'edit_time', 'edit_source', 'edit_notes',
    'term_type_option', 'term_type_value', 'term_type_text', 'term_type_language',
    'status_option', 'status_value', 'status_text', 'status_language',
    'broader_linkref', 'broader_linkfield', 'broader_linkreffield', 'broader_linkdb', 'broader_term',
    'narrower_linkref', 'narrower_linkfield', 'narrower_linkreffield', 'narrower_linkdb', 'narrower_term_number','narrower_term',  # % nieuw
    'used_for_linkref', 'used_for_linkfield', 'used_for_linkreffield', 'used_for_linkdb', 'used_for_term'
]

# Select only columns that exist in the DataFrame
df = df[[col for col in final_columns if col in df.columns]]

df


,priref,use_count,main_term,term_number,source,source_link,scope_note,input_date,input_name,input_time,...,narrower_linkfield,narrower_linkreffield,narrower_linkdb,narrower_term_number,narrower_term,used_for_linkref,used_for_linkfield,used_for_linkreffield,used_for_linkdb,used_for_term
0,1,19753,Nederland,None,None,None,None,None,None,None,...,NaN,NaN,NaN,479,Ridderkerk,NaN,NaN,NaN,NaN,NaN
1,1,19753,Nederland,None,None,None,None,None,None,None,...,NaN,NaN,NaN,23,Rotterdam,NaN,NaN,NaN,NaN,NaN
2,1,19753,Nederland,None,None,None,None,None,None,None,...,NaN,NaN,NaN,25737,Schoonloo,NaN,NaN,NaN,NaN,NaN
3,1,19753,Nederland,None,None,None,None,None,None,None,...,NaN,NaN,NaN,1522,Sint-Michielsgestel,NaN,NaN,NaN,NaN,NaN
4,1,19753,Nederland,None,None,None,None,None,None,None,...,NaN,NaN,NaN,25414,Uitdam,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222323,25865,None,kleitablet,None,Van Dale woordenboek,None,"plaat van klei, gedroogd of gebakken, m.n. bes...",2025-03-21,cmulders,11:44:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2222324,25865,None,kleitablet,None,Van Dale woordenboek,None,"plaat van klei, gedroogd of gebakken, m.n. bes...",2025-03-21,cmulders,11:44:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2222325,25865,None,kleitablet,None,Van Dale woordenboek,None,"plaat van klei, gedroogd of gebakken, m.n. bes...",2025-03-21,cmulders,11:44:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2222326,25865,None,kleitablet,None,Van Dale woordenboek,None,"plaat van klei, gedroogd of gebakken, m.n. bes...",2025-03-21,cmulders,11:44:15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data analysis

### 1. Een overzicht van duidelijke fouten die direct te herstellen zijn in de complete collectie (jaartallen, spelfouten, data types etc.)

In [127]:
# # Filter data
# cols = ['priref', # nummer term in systeem
#  'use_count', # aantal keer term gebruikt
#  'main_term', # prefLabel term
#  'term_number', # AAT URI i.e. 300000240
#  'source', # Thesaurus, i.e. AAT, Wikipedia
#  'source_link', # AAT link
#  'input_name', # wie term heeft toegevoegd
#  'input_source', # waar/ welke collectie het is ingevoerd
#  'narrower_term_lrefs', # nummer narrower term in systeem
#  'broader_term_lrefs', # nummer broader term in systeem
#  'used_for_lrefs', # altLabel, is wel een eigen term in het systeem.
#  'edit_name', # name of the editor
#  'edit_source', # waar de edit heeft plaatsgevonden i.e. thesau
#  'term_type_option', # iets van een categorie binnen de systeem, zoals DIM voor afmeting/dimension
#  'term_type_value', # categorie waarin term onder valt
#  'term_type_text', # in meerdere talen de categorienaam
#  'status_option', # nummer behorend tot de status van een term, bijv. 3=kandidaat
#  'status_value', # label van de status van een term, zoals kandidaat of niet gedefinieerd
#  'status_text', # meerdere vertalingen van de status
#  'status_language', # nummer van de taal i.e. 0 = engels, van de status label
#  'broader_linkref', # systeem id van de broader term
#  'broader_linkfield', # type broader term (is altijd term)
#  'broader_linkreffield', # veldnaam dat aangeeft dat het om een brader term gaat
#  'broader_linkdb', # path in addlib, soort opslag pad
#  'broader_term', # prefLabel broader term
#  'used_for_linkref', # nummer van altLabel term
#  'used_for_linkfield', # wat voor type dat bovenste is (=term)
#  'used_for_linkreffield', #categorie naam (=term)
#  'used_for_linkdb', # # path in addlib, soort opslag pad
#  'used_for_term'] # altLabel main term

# df_termen = df.dropna(subset=cols, how='all')

In [128]:
# df_termen_vs_broader['source_link'].nunique() # 279

## Functies

In [129]:

def bevat_cijfer_teken_of_1letter(term):
    if pd.isna(term):
        return 'nee'

    # Check op cijfer
    if re.search(r'\d', term):
        return 'ja'
    
    # Check op speciaal teken (niet letter of spatie)
    if re.search(r'[^\w\s]', term):
        return 'ja'
    
    # Check op 1-letterwoorden
    woorden = term.strip().split()
    if any(len(w) == 1 for w in woorden):
        return 'ja'
    
    return 'nee'


In [130]:
def build_fuzzy_index(term_list):
    term_map = {t.lower(): t for t in term_list if isinstance(t, str)}
    term_dict = defaultdict(list)
    for t_lower in term_map.keys():
        term_dict[len(t_lower)].append(t_lower)
    return term_map, term_dict

In [131]:
def fuzzy_match_with_flag(term, target_dict, target_map, threshold=85, exclude_exact=True):
    """
    Retourneert:
    - 'ja' of 'nee' afhankelijk van of er matches zijn
    - string met: originele_term, match1, match2
    
    Parameters:
    - exclude_exact: als True, sluit exacte matches (zelfde lowercase string) uit
    """
    if pd.isna(term):
        return 'nee', ''
    
    term_orig = str(term)
    term_lower = term_orig.lower().strip()
    term_len = len(term_lower)

    mogelijke_matches = (
        target_dict.get(term_len, []) +
        target_dict.get(term_len - 1, []) +
        target_dict.get(term_len + 1, [])
    )

    matches = [
        target_map[t]
        for t in mogelijke_matches
        if (not exclude_exact or t != term_lower) and ratio(term_lower, t) > threshold
    ]

    if matches:
        return 'ja', f"{term_orig}, {', '.join(matches)}"
    else:
        return 'nee', ''


## Toepassingen

In [156]:
# Filter data
df_termen_vs_broader = df[
    ['priref','use_count','main_term','term_number','source','source_link','term_type_option','term_type_value','status_value',
     'broader_term_lrefs','broader_term',
     'narrower_term_lrefs','narrower_term_number','narrower_term']
     ]
df_termen_vs_broader = df_termen_vs_broader.drop_duplicates()

##### Kolom toevoegen `broader_term_uppercase`.

In [157]:
df_termen_vs_broader['broader_term_uppercase'] = df_termen_vs_broader['broader_term'].apply(
    lambda term: 'ja' if isinstance(term, str) and term.isupper() else 'nee'
)

##### Kolom toevoegen `main_term_mist_uri` en `meerdere_uris_main_term`.

In [158]:
# Check of URI ontbreekt
df_termen_vs_broader['main_term_mist_uri'] = df_termen_vs_broader['source_link'].apply(
    lambda x: 'ja' if pd.isna(x) or x == '' else 'nee'
)

# Groeperen er uri en als er >2 zijn worden ze toegevoegd. 
uri_mapping = (
    df_termen_vs_broader
    .loc[df_termen_vs_broader['main_term_mist_uri'] == 'nee']
    .groupby('main_term')['source_link']
    .agg(lambda x: ', '.join(sorted(set(x))) if len(set(x)) > 1 else '')
    .to_dict()
)

# Voeg toe aan kolom, leeg als geen meerdere URIs
df_termen_vs_broader['meerdere_uris_main_term'] = df_termen_vs_broader['main_term'].map(uri_mapping)


##### Kolom toevoegen `meerdere_categorien_main_term`

In [159]:
#----------Zijn er termen die tot meerdere categorien behoren? i.e. land en provincie?------
unique_term_categorie = df_termen_vs_broader.groupby('main_term')['term_type_value'].nunique()
df_termen_vs_broader['meerdere_categorien_main_term'] = df_termen_vs_broader['main_term'].map(unique_term_categorie).gt(1).map({True:'ja', False: 'nee'})

##### Kolom toevoegen `meerdere_spelling_main_term` en `gevonden_matches_main_in_main`.

In [160]:
main_term_list = df_termen_vs_broader['main_term'].dropna().unique()
main_term_map, main_term_dict = build_fuzzy_index(main_term_list)

resultaten_main = df_termen_vs_broader['main_term'].apply(
    lambda t: fuzzy_match_with_flag(t, main_term_dict, main_term_map)
)

df_termen_vs_broader[['meerdere_spelling_main_term', 'gevonden_matches_main_in_main']] = pd.DataFrame(
    resultaten_main.tolist(), index=df_termen_vs_broader.index
)

##### Kolom toevoegen `broader_term_in_main_term` en `broader_terms_match_main_term`.

In [161]:
resultaten_broader_to_main = df_termen_vs_broader['broader_term'].apply(
    lambda t: fuzzy_match_with_flag(t, main_term_dict, main_term_map, threshold=80)
)

df_termen_vs_broader[['broader_term_in_main_term', 'broader_terms_match_main_term']] = pd.DataFrame(
    resultaten_broader_to_main.tolist(), index=df_termen_vs_broader.index
)

##### Kolom toevoegen `meerdere_spelling_broader_in_broader` en `gevonden_spellingen_broader_in_broader`.

In [162]:
broader_term_list = df_termen_vs_broader['broader_term'].dropna().unique()
broader_term_map, broader_term_dict = build_fuzzy_index(broader_term_list)

resultaten_broader_self = df_termen_vs_broader['broader_term'].apply(
    lambda t: fuzzy_match_with_flag(t, broader_term_dict, broader_term_map, exclude_exact=True)
)

df_termen_vs_broader[['meerdere_spelling_broader_in_broader', 'gevonden_spellingen_broader_in_broader']] = pd.DataFrame(
    resultaten_broader_self.tolist(), index=df_termen_vs_broader.index
)


##### Kolom toevoegen `main_term_bevat_tekens_of_cijfers`.

In [163]:
df_termen_vs_broader['main_term_bevat_tekens_of_cijfers'] = df_termen_vs_broader['main_term'].apply(bevat_cijfer_teken_of_1letter)


##### Kolom toevoegen `narrower_in_main_term` en `gevonden_matches_narrower_in_main`.

In [164]:
resultaten_narrower = df_termen_vs_broader['narrower_term'].apply(
    lambda t: fuzzy_match_with_flag(t, main_term_dict, main_term_map, exclude_exact=False)
)
df_termen_vs_broader[['narrower_in_main_term', 'gevonden_matches_narrower_in_main']] = pd.DataFrame(resultaten_narrower.tolist(), index=df_termen_vs_broader.index)


#### Write to Excel

In [ ]:
# order van kolommen sorteren

In [142]:
df_termen_vs_broader.columns

Index(['priref', 'use_count', 'main_term', 'term_number', 'source',
       'source_link', 'term_type_value', 'status_value', 'broader_term_lrefs',
       'broader_term', 'narrower_term_lrefs', 'narrower_term_number',
       'narrower_term', 'broader_term_uppercase', 'main_term_mist_uri',
       'meerdere_uris_main_term', 'meerdere_categorien_main_term',
       'meerdere_spelling_main_term', 'gevonden_matches_main_in_main',
       'broader_term_in_main_term', 'broader_terms_match_main_term',
       'meerdere_spelling_broader_in_broader',
       'gevonden_spellingen_broader_in_broader',
       'bevat_tekens_of_cijfers_in_main_term', 'narrower_in_main_term',
       'gevonden_matches_narrower_in_main'],
      dtype='object')

In [167]:
order_kolommen = ['priref', 'use_count', 'main_term', 'term_number', 'source',
       'source_link','term_type_option', 'term_type_value', 'status_value', 'main_term_mist_uri',
       'meerdere_uris_main_term', 'meerdere_categorien_main_term',
       'meerdere_spelling_main_term', 'gevonden_matches_main_in_main','main_term_bevat_tekens_of_cijfers',

       'broader_term_lrefs','broader_term', 'broader_term_uppercase', 
       'broader_term_in_main_term', 'broader_terms_match_main_term',
       'meerdere_spelling_broader_in_broader','gevonden_spellingen_broader_in_broader',
       
       'narrower_term_lrefs', 'narrower_term_number',
       'narrower_term','narrower_in_main_term','gevonden_matches_narrower_in_main']

In [169]:
df_termen_vs_broader = df_termen_vs_broader[order_kolommen]

In [170]:
with pd.ExcelWriter("kc_datakwaliteit.xlsx", engine='openpyxl', mode='w') as writer:
    df_termen_vs_broader.to_excel(writer, sheet_name='termen_kwaliteitscheck', index=False)

### 2. Een overzicht van de Axiell thesaurus termen, met N objecten die hieraan gekoppeld zijn en de velden waarin ze voorkomen.

Daarbij ligt nadruk op termen die niet dubbelzinnig geïnterpreteerd kunnen worden door het voorkomen ervan in meerdere velden: ‘Utrecht’ in veld provincie, gemeente etc. blijven we af, maar dat moet wel inzichtelijk zijn in de analyse.  

- N-termen komen als `prefLabel` alleen voor in 'main_term' veld en 'narrower_term' veld.

In [ ]:
# pip install rapidfuzz

In [ ]:
# # check of een N-term ergens anders in een kolom voorkomt, alles .tolower, een kolom met waar het fout staat maar wel hetzelfde is wordt later toegevoegd.

# from rapidfuzz import fuzz

# # Verzamel alle andere termen in de DataFrame (zonder 'narrower_term'), als lowercase strings
# alle_waarden = (
#     df_termen_vs_broader.drop(columns=['narrower_term'])
#       .astype(str)
#       .stack()
#       .str.lower()
#       .dropna()
#       .unique()
# )

# # Functie om te checken of een string fuzzy matched met iets uit de lijst
# def is_fuzzy_match(zoekterm, lijst, threshold=90):
#     if pd.isna(zoekterm):
#         return False
#     zoekterm = str(zoekterm).lower()
#     return any(fuzz.ratio(zoekterm, str(term)) >= threshold for term in lijst)

# # Nieuwe kolom met fuzzy matches
# df_termen_vs_broader['narrower_bestaat_elders'] = df_termen_vs_broader['narrower_term'].apply(
#     lambda x: 'ja' if is_fuzzy_match(x, alle_waarden, threshold=90) else 'nee'
# )


In [ ]:
# # check of een N-term ergens anders in een kolom voorkomt, alles .tolower, een kolom met waar het fout staat maar wel hetzelfde is wordt later toegevoegd.

# from rapidfuzz import fuzz

# # Verzamel alle andere termen in de DataFrame (zonder 'narrower_term'), als lowercase strings
# alle_waarden = (
#     df.drop(columns=['narrower_term'])
#       .astype(str)
#       .stack()
#       .str.lower()
#       .dropna()
#       .unique()
# )

# # Functie om te checken of een string fuzzy matched met iets uit de lijst
# def is_fuzzy_match(zoekterm, lijst, threshold=90):
#     if pd.isna(zoekterm):
#         return False
#     zoekterm = str(zoekterm).lower()
#     return any(fuzz.ratio(zoekterm, str(term)) >= threshold for term in lijst)

# # Nieuwe kolom met fuzzy matches
# df['narrower_bestaat_elders'] = df['narrower_term'].apply(
#     lambda x: 'ja' if is_fuzzy_match(x, alle_waarden, threshold=90) else 'nee'
# )


In [ ]:
# cols_nrw = [
#  'priref', # nummer term in systeem
#  'use_count', # aantal keer term gebruikt
#  'main_term', # prefLabel term
#  'source_link', # AAT link
#  'narrower_term_number',
#  'narrower_term',
#  'narrower_bestaat_elders'
# ]

# df_termen_vs_narrower = df[cols_nrw].drop_duplicates().reset_index(drop=True)

In [ ]:
# df_termen_vs_broader = (
#     df_termen_vs_broader
#     .assign(
#         narrower_als_main_term=lambda x: x['narrower_term'].str.lower().isin(x['main_term'].str.lower()).map({True: 'ja', False: 'nee'})
#     )
# )

In [ ]:
# resultaten_narrower = df_termen_vs_broader['narrower_term'].apply(
#     lambda t: fuzzy_match_with_flag(t, main_term_dict, main_term_map)
# )

# # Stap 3: Opslaan in nieuwe kolommen
# df_termen_vs_broader[['narrower_in_main_term', 'gevonden_matches_narrower_in_main']] = pd.DataFrame(
#     resultaten_narrower.tolist(), index=df_termen_vs_broader.index
# )

In [ ]:
# resultaten_narrower = df_termen_vs_broader['narrower_term'].apply(
#     lambda t: fuzzy_match_with_flag(t, main_term_dict, main_term_map, exclude_exact=False)
# )
# df_termen_vs_broader[['narrower_in_main_term', 'gevonden_matches_narrower_in_main']] = pd.DataFrame(resultaten_narrower.tolist(), index=df_termen_vs_broader.index)


## Save to excel

In [ ]:
# # Als de sheet geupdated wordt, kan het eerst verwijderd worden met deze code.
# from openpyxl import load_workbook

# file_path = "kc_datakwaliteit.xlsx"
# sheet_name = "narrowers_kwaliteitscheck"

# # Load workbook and remove the sheet if it exists
# book = load_workbook(file_path)
# if sheet_name in book.sheetnames:
#     book.remove(book[sheet_name])
#     book.save(file_path)


In [ ]:
# with pd.ExcelWriter("kc_datakwaliteit.xlsx", engine='openpyxl', mode='a') as writer:
#     df_termen_vs_narrower.to_excel(writer, sheet_name='narrowers_kwaliteitscheck', index=False)

Resultaat en wat opvalt:

- source link naar thesaurus is niet altijd de uri: in data <http://browser.aat-ned.nl/300000240> en uri <http://vocab.getty.edu/aat/300000240>

- termen hebben soms dezelfde label maar andere status en gelinkte properties:
`paralmoer` - `used for linkref` -> `paarlemoer`

### 3. Afgeleid daarvan een inschatting van de werkvoorraad m.b.t. ‘no regret’ termen en trefwoorden die te verbeteren zijn en (wat mij betreft) te koppelen aan URI’s. Focus ligt (dacht ik?) op termen m.b.t. onderwerp, locatie, materiaal, techniek en vervaardigers.

note to self: kijk in de data naar welke categorien er allemaal zijn, op basis daarvan mogelijk meenemen, met de kolommen waar er van alles nog moet gebeuren